In [229]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


In [177]:
df= pd.read_csv('C:/Users/avhad/Downloads/all_season_details .csv')

C:\Users\avhad\AppData\Local\Temp\ipykernel_23568\1761648653.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('C:/Users/avhad/Downloads/all_season_details .csv')


In [178]:
df

,comment_id,season,match_id,match_name,home_team,away_team,current_innings,innings_id,over,ball,...,wicket_id,wkt_batsman_name,wkt_bowler_name,wkt_batsman_runs,wkt_batsman_balls,wkt_text,isRetiredHurt,text,preText,postText
0,110,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"nice and full, angling and perhaps swinging in...",<p><strong>7.30pm</strong> Gaikwad and Conway ...,NaN
1,120,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"Conway shuffles across off, and shimmies out a...",NaN,NaN
2,130,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"good length, angling in at off and then straig...",NaN,NaN
3,140,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"shorter and slanting into middle and leg, Gaik...","<p>Rummy: ""Fast bowling options for CSK defini...",NaN
4,150,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Shami hits a heavy length and brings this back...,"<p>LG: ""Watching out for Hangargekar. He was i...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242545,219020,2008.0,336040,CSK v RR,CSK,RR,RR,2,20,2,...,NaN,NaN,NaN,NaN,NaN,NaN,False,lovely! Balaji's bowled a yorker outside off s...,NaN,NaN
242546,219030,2008.0,336040,CSK v RR,CSK,RR,RR,2,20,3,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Warne steps down the track and throws his bat ...,NaN,<p>6 off 3 balls.</p>
242547,219050,2008.0,336040,CSK v RR,CSK,RR,RR,2,20,4,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Warne reaches out for a full ball and squeezes...,NaN,<p>Three to win off two balls.</p>
242548,219040,2008.0,336040,CSK v RR,CSK,RR,RR,2,20,4,...,NaN,NaN,NaN,NaN,NaN,NaN,False,what has he done! Balaji has bowled a wide out...,<p>Rajasthan need two runs a ball. Balaji from...,NaN


In [181]:
df.drop(['comment_id','shortText','text','preText','postText'],axis=1,inplace=True)

In [183]:
df.loc[(df['match_id'].isin([1359496,1359538])) & (df['season'].isnull()), 'season'] = 2023

In [185]:
df.drop_duplicates(inplace=True)

In [186]:
data= df[['season', 'match_id', 'home_team', 'away_team', 'current_innings','innings_id', 'over', 'ball', 'runs', 'wicket_id','wkt_batsman_name', 'wkt_batsman_runs']].copy()
data['cumulative_runs'] = data.groupby(['match_id', 'current_innings'])['runs'].cumsum()
data['run_rate'] = data['cumulative_runs'] / (data['over'] + 0.1)
data['wickets_lost'] = data.groupby(['match_id', 'current_innings'])['wkt_batsman_runs']\
                                            .transform(lambda x: (x >= 0).cumsum())
data.fillna(0, inplace=True)



In [188]:
# Create a dictionary for mapping
team_mapping = {'GT': 0, 'PBKS': 1, 'LSG': 2, 'SRH': 3,'RCB': 4, 'CSK': 5, 'DC': 6, 'RR': 7,
                'KKR': 8, 'MI': 9, 'KXIP': 10, 'RPS': 11,'GL': 12, 'PWI': 13, 'KOCHI': 14}
# Apply the mapping to 'home_team' and 'away_team'
data['home_team_encoded'] = data['home_team'].map(team_mapping)
data['away_team_encoded'] = data['away_team'].map(team_mapping)
data['current_innings'] = data['current_innings'].map(team_mapping)
data.fillna(0, inplace=True)

In [191]:
X = data[['home_team_encoded','away_team_encoded','current_innings','over', 'ball', 'run_rate','wickets_lost']]
y = data['cumulative_runs']

In [193]:
X

,home_team_encoded,away_team_encoded,current_innings,over,ball,run_rate,wickets_lost
0,0.0,5.0,5.0,1,1,0.000000,0
1,0.0,5.0,5.0,1,2,0.909091,0
2,0.0,5.0,5.0,1,3,0.909091,0
3,0.0,5.0,5.0,1,4,1.818182,0
4,0.0,5.0,5.0,1,5,1.818182,0
...,...,...,...,...,...,...,...
242545,5.0,7.0,7.0,20,2,7.810945,7
242546,5.0,7.0,7.0,20,3,7.860697,7
242547,5.0,7.0,7.0,20,4,7.910448,7
242548,5.0,7.0,7.0,20,4,8.009950,7


In [195]:
y

0           0
1           1
2           1
3           2
4           2
         ... 
242545    157
242546    158
242547    159
242548    161
242549    163
Name: cumulative_runs, Length: 242549, dtype: int64

In [197]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [199]:
X_train

,home_team_encoded,away_team_encoded,current_innings,over,ball,run_rate,wickets_lost
199033,13.0,8.0,13.0,12,4,5.537190,3
237647,6.0,3.0,6.0,17,1,8.596491,2
10462,0.0,6.0,6.0,1,5,4.545455,1
39476,7.0,4.0,7.0,8,4,8.518519,0
137929,3.0,5.0,3.0,9,2,6.153846,2
...,...,...,...,...,...,...,...
119880,8.0,3.0,3.0,10,6,7.029703,3
103695,3.0,9.0,3.0,12,4,8.016529,1
131933,10.0,4.0,4.0,6,1,8.196721,2
146868,8.0,3.0,3.0,18,6,7.182320,5


In [201]:
X_test

,home_team_encoded,away_team_encoded,current_innings,over,ball,run_rate,wickets_lost
122567,7.0,9.0,9.0,10,5,4.455446,4
147657,9.0,5.0,9.0,5,2,6.078431,0
27100,0.0,3.0,0.0,1,1,0.000000,0
42494,5.0,9.0,5.0,2,5,0.952381,2
154360,3.0,10.0,3.0,19,1,5.863874,5
...,...,...,...,...,...,...,...
19878,0.0,6.0,0.0,17,5,8.362573,3
217200,5.0,6.0,5.0,14,3,9.503546,3
188030,8.0,7.0,7.0,3,1,4.516129,0
220381,8.0,9.0,8.0,15,1,6.092715,2


In [203]:
rf_model=RandomForestRegressor(n_estimators=200,max_depth=None,random_state=42)

In [205]:
rf_model.fit(X_train,y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [206]:
y_pred=rf_model.predict(X_test)

In [207]:
print('RandomForestRegressor')
rf_simple_mse=mean_squared_error(y_test,y_pred)
rf_r2=r2_score(y_test,y_pred)
print('MSE score: ',rf_simple_mse)
print('R2 score: ',rf_r2)

RandomForestRegressor
MSE score:  0.06308769274376419
R2 score:  0.9999747216866168


In [208]:
new_data = pd.DataFrame({
    'home_team_encoded': [4],
    'away_team_encoded': [6],
    'current_innings':[1],
    'over': [20],  
    'ball': [2],      
    'run_rate': [7],        
    'wickets_lost': [4]          
})

In [209]:
new_data 

,home_team_encoded,away_team_encoded,current_innings,over,ball,run_rate,wickets_lost
0,4,6,1,20,2,7,4


In [210]:
predict_runs=rf_model.predict(new_data)
print(f"Predicted Cumulative Runs: {predict_runs[0]}")

Predicted Cumulative Runs: 141.0


In [211]:
print('Polynomial  regression')

poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
pred = poly_model.predict(X_test_poly)
pr_mse=mean_squared_error(y_test,pred)
print('MSE score: ',pr_mse)
pr_r2=r2_score(y_test,pred)
print('R2 score: ',pr_r2)

Polynomial  regression
MSE score:  4.074348638708838e-24
R2 score:  1.0


In [212]:
lr_model = LinearRegression()

In [213]:
lr_model.fit(X_train,y_train)

LinearRegression()

In [214]:
pred=lr_model.predict(X_test)
pred

array([ 62.53986544,  35.28701379, -18.54705613, ...,  12.3056283 ,
       110.42547893,  50.89679448])

In [215]:
print('Linear  regression')
lr_mse=mean_squared_error(y_test,pred)
print('MSE score: ',lr_mse)
lr_r2=r2_score(y_test,pred)
print('R2 score: ',lr_r2)

Linear  regression
MSE score:  266.7656880261987
R2 score:  0.8931109005809656


In [216]:
print("Gradient Boosting Regressor:")
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
mse_gb = mean_squared_error(y_test, y_pred_gb)
print("MSE score: ",mse_gb)
r2_gb = r2_score(y_test, y_pred_gb)
print("R2 score: ",r2_gb)

Gradient Boosting Regressor:
MSE score:  3.4692460767645668
R2 score:  0.9986099239690377
